In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

from data import get_dataloaders
from training import train_loop, evaluate
from unet import UNet, UNet6, ConvNextUNet, UNetWithoutDEM
from metrics import Precision, Recall, HardIoU
from loss import iou_loss, bce_loss, dice_loss

dataloaders = get_dataloaders("dataset")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def evaluate_method(model_cls, loss_fn, directory):
    num_epochs = 100
    model = model_cls(3, 1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), 0.001)
    metrics = {"precision": Precision(), "recall": Recall(), "iou": HardIoU()}

    train_loop(dataloaders, model, loss_fn, optimizer,
        num_epochs=num_epochs, metrics=metrics, early_stop=10, directory=directory, best_metric="iou")
    
    loss, computed_metrics = evaluate(dataloaders[2], model, loss_fn, metrics)
    print(f"test loss: {loss}")
    for key, metric in computed_metrics.items():
        print(f"{key}: {metric}")

    df = pd.read_csv(f"{directory}/metrics.csv")
    plt.figure()
    plt.plot(df["epoch"], df["train_loss"])
    plt.plot(df["epoch"], df["val_loss"])
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    plt.figure()
    plt.plot(df["epoch"], df["iou"])
    plt.xlabel("Epoch")
    plt.ylabel("IoU")
    plt.show()

In [ ]:
evaluate_method(UNet, iou_loss, "unet")
evaluate_method(UNetWithoutDEM, iou_loss, "unet_without_dem")

evaluate_method(UNet, bce_loss, "unet_bce")
evaluate_method(UNet, dice_loss, "unet_dice")

evaluate_method(UNet6, iou_loss, "unet6")
evaluate_method(ConvNextUNet, iou_loss, "unet_convnext")